# Custom Object Detection
## Data Preprocessor

This notebook is for preprocessing your data into tf record files and creating a label_map

Project Structure of Collected
```
data_preprocessing/  
    |
    |---data/
    |       |---predefined_classes.txt
    |       |___train/
    |       |       |---all train images files
    |       |       |---...
    |       |       |---xml files of respective test images
    |       |       |---...
    |       |
    |       |___test/
    |               |---all test images files
    |               |---...
    |               |---xml files of respective test images
    |               |---....
    |           
    |---labelImg.exe   
    |---transform_resolution.py
    |---xml_to_csv.py
    |---generate_tfrecord.py
    |---Preprocess_Data.ipynb
```

## Transform Resolution (optional)
If you manually shot the images using a camera or phone, you may want to reduce the resolution

In [ ]:
# resize all images in train and test dir to 800x600
!python transform_resolution.py -d data/train/ -s 800 600

!python transform_resolution.py -d data/test/ -s 800 600

## Convert xml to csv files

In [ ]:
!python xml_to_csv.py -i data/train -o data/train_labels.csv

!python xml_to_csv.py -i data/test -o data/test_labels.csv

## Generate tf.record files from csv
Edit `generate_tfrecord.py` and update your labels in `class_text_to_int()` function at line 31. Add if statements as per your labels.

For example, if we have a dog and cat detector, we would modify the function as
```python
def class_text_to_int(row_label):
    if row_label == 'cat':
        return 1
    elif row_label == 'dog':
        return 2
    else:
        None
```

In [ ]:
%%writefile generate_tfrecord.py
"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=train.record

  # Create test data:
  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('image_dir', '', 'Path to images')
FLAGS = flags.FLAGS


# REPLACE/ADD IF STATEMENTS AS PER YOUR YOUR LABELS
def class_text_to_int(row_label):
    if row_label == 'product':
        return 1
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(FLAGS.image_dir)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    #print(grouped)
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.app.run()


Once you are done with updating the file with your labels, run the script with appropriate directories of images and csv files

In [ ]:
!python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=data/train.record --image_dir=data/train

!python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=data/test.record --image_dir=data/test

## Creating the Label Map
In the below cell edit the label id and name and add item objects based on your labels

For example,
```
item {
    id: 1
    name: 'cat'
}
item {
    id: 2
    name: 'dog'
}
```

In [ ]:
%%writefile data/label_map.pbtxt

item {
    id: 1
    name: 'product'
}

## Archiving the processed data files
Now that we have generated the required files for training like the `label_map.pbtxt`, `train.record` and `test.record`, we will archive it into a zip file so that we can easily upload it to our [training notebook on colab]().

In [ ]:
import os
import shutil

In [ ]:
os.mkdir('data/tmp/')

shutil.copy("data/label_map.pbtxt", "data/tmp/label_map.pbtxt")
shutil.copy("data/train.record", "data/tmp/train.record")
shutil.copy("data/test.record", "data/tmp/test.record")

shutil.make_archive('data.zip', 'zip', 'data/tmp')

shutil.rmtree('data/tmp/')

#### Now you can move on to the [colab training notebook](https://colab.research.google.com/github/theneuralbeing/object_detection_template/blob/master/object_detection_training.ipynb) for further instructions